In [41]:
import random

In [42]:
def new_word():
    with open('words_250000_train.txt', 'r') as f:
        words = f.readlines()
        return random.choice(words).strip()

In [43]:
def play():
    state1=[-2]*50
    state2=[0]*26
    num_misses=0
    word=new_word()
    for i in range(len(word)):
        state1[i]=-1
    while(num_misses<6):
        print("Guess the word: ",end='')
        for i in range(len(word)):
            if(state1[i]==-1):
                print("_",end=' ')
            else:
                print(chr(state1[i]+ord('a')),end=' ')
        print()
        guess=guess_letter(state1,state2)
        if(guess in word and state2[ord(guess)-ord('a')]==0):
            state2[ord(guess)-ord('a')]=1
            for i in range(len(word)):
                if(word[i]==guess):
                    state1[i]=ord(guess)-ord('a')
            flag=True
            for i in range(len(word)):
                if(state1[i]==-1):
                    flag=False
                    break
            if(flag):
                print("You won! The word was "+word)
                return
        else:
            num_misses+=1
            print("Missed, "+guess+" was wrong, mistake "+str(num_misses)+" out of 6.")
            if(state2[ord(guess)-ord('a')]==0):
                state2[ord(guess)-ord('a')]=-1
            if(num_misses==6):
                print("You lost! The word was "+word)
                return

In [44]:
def guess_letter(state1,state2):
    return input()[0]

In [45]:
play()

Guess the word: _ _ _ _ _ _ _ _ _ _ 


IndexError: string index out of range

In [ ]:
import random
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# Load words from words.txt into a list
with open('words_250000_train.txt', 'r') as f:
    words_list = f.read().splitlines()

# Hangman class
class Hangman:
    def __init__(self, word):
        print('New word: {}'.format(word))
        self.word = word
        self.guesses = []
        self.max_attempts = 6
        self.attempts_left = self.max_attempts

    def guess(self, letter):
        self.guesses.append(letter)
        if letter not in self.word:
            self.attempts_left -= 1

    def is_word_guessed(self):
        return all(letter in self.guesses for letter in self.word)

    def get_current_state(self):
        return ''.join([letter if letter in self.guesses else '_' for letter in self.word])

    def get_remaining_attempts(self):
        return self.attempts_left

    def is_game_over(self):
        return self.attempts_left == 0 or self.is_word_guessed()

# DQN Agent
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = []
        self.gamma = 0.9  # discount factor
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        q_values = self.model.predict(state)
        return np.argmax(q_values[0])

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma * np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

# RL Training
def train_rl_agent(words_list, epochs=100, batch_size=32):
    # Create a dictionary for indexing characters for actions
    chars = sorted(list(set(''.join(words_list))))
    char_to_idx = {char: idx for idx, char in enumerate(chars)}
    idx_to_char = {idx: char for char, idx in char_to_idx.items()}

    state_size = len(chars)
    action_size = len(chars)

    agent = DQNAgent(state_size, action_size)

    for epoch in range(epochs):
        word = random.choice(words_list)
        hangman = Hangman(word)
        state = np.zeros(state_size)
        done = False

        while not done:
            action = agent.act(state.reshape(1, -1))
            letter = idx_to_char[action]
            reward = -1 if letter not in word else -100000 if letter in hangman.guesses else 0
            hangman.guess(letter)
            next_state = np.array([1 if idx_to_char[idx] in hangman.guesses else 0 for idx in range(state_size)])
            done = hangman.is_game_over()
            if done:
                reward = 100 if hangman.is_word_guessed() else -1000

            agent.remember(state.reshape(1, -1), action, reward, next_state.reshape(1, -1), done)
            state = next_state

        if epoch > batch_size:
            agent.replay(batch_size)

        print(f"Epoch: {epoch + 1}/{epochs}, Word: {word}, Epsilon: {agent.epsilon}")

    return agent

if __name__ == "__main__":
    # Training the RL agent
    trained_agent = train_rl_agent(words_list)

    # Save the trained agent's model
    trained_agent.model.save('hangman_dqn_model.h5')


New word: lenvoi
Epoch: 1/100, Word: lenvoi, Epsilon: 1.0
New word: pigeoneer
Epoch: 2/100, Word: pigeoneer, Epsilon: 1.0
New word: selffondest
Epoch: 3/100, Word: selffondest, Epsilon: 1.0
New word: preindemnification
Epoch: 4/100, Word: preindemnification, Epsilon: 1.0
New word: corybant
Epoch: 5/100, Word: corybant, Epsilon: 1.0
New word: compressionignition
Epoch: 6/100, Word: compressionignition, Epsilon: 1.0
New word: unificationist
Epoch: 7/100, Word: unificationist, Epsilon: 1.0
New word: unaristocratically
Epoch: 8/100, Word: unaristocratically, Epsilon: 1.0
New word: playreader
Epoch: 9/100, Word: playreader, Epsilon: 1.0
New word: ductus
Epoch: 10/100, Word: ductus, Epsilon: 1.0
New word: ephemerous
Epoch: 11/100, Word: ephemerous, Epsilon: 1.0
New word: hugelia
Epoch: 12/100, Word: hugelia, Epsilon: 1.0
New word: undull
Epoch: 13/100, Word: undull, Epsilon: 1.0
New word: oscinis
Epoch: 14/100, Word: oscinis, Epsilon: 1.0
New word: pretrying
Epoch: 15/100, Word: pretrying, E

In [ ]:
!pip install keras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.5 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0.1 -> 23.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [ ]:
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 MB 6.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 9.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 9.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.3/400.3 kB 8.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 9.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 8.4 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 8.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.8/181.8 kB 7.1 MB/s eta 0:00:00
  Using cached google_auth_oauthlib-1.0.0-py2.py3-non

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('hangman_dqn_model.h5')

# Dictionary for indexing characters for actions
chars = sorted(list(set('abcdefghijklmnopqrstuvwxyz')))
char_to_idx = {char: idx for idx, char in enumerate(chars)}
idx_to_char = {idx: char for char, idx in char_to_idx.items()}

# Hangman class
class Hangman:
    def __init__(self, word):
        self.word = word
        self.guesses = []
        self.max_attempts = 6
        self.attempts_left = self.max_attempts

    def guess(self, letter):
        self.guesses.append(letter)
        if letter not in self.word:
            self.attempts_left -= 1

    def is_word_guessed(self):
        return all(letter in self.guesses for letter in self.word)

    def get_current_state(self):
        return ''.join([letter if letter in self.guesses else '_' for letter in self.word])

    def get_remaining_attempts(self):
        return self.attempts_left

    def is_game_over(self):
        return self.attempts_left == 0 or self.is_word_guessed()

# Function to play Hangman using the trained DQN model
def play_hangman():
    word = random.choice(words_list)
    hangman = Hangman(word)
    state_size = len(chars)
    state = np.zeros(state_size)

    while not hangman.is_game_over():
        # Get the action (letter) from the DQN model
        action = model.predict(state.reshape(1, -1))
        action_idx = np.argmax(action)
        letter = idx_to_char[action_idx]

        print(f"Guessing: {letter}")
        hangman.guess(letter)
        state = np.array([1 if idx_to_char[idx] in hangman.guesses else 0 for idx in range(state_size)])
        print(f"Current State: {hangman.get_current_state()}")
        print(f"Attempts Left: {hangman.get_remaining_attempts()}")

    if hangman.is_word_guessed():
        print(f"Congratulations! You guessed the word: {word}")
    else:
        print(f"Game Over! The word was: {word}")

if __name__ == "__main__":
    play_hangman()


1/1 [==============================] - 0s 116ms/step
Guessing: c
Current State: _______c___
Attempts Left: 6
1/1 [==============================] - 0s 43ms/step
Guessing: c
Current State: _______c___
Attempts Left: 6
1/1 [==============================] - 0s 16ms/step
Guessing: c
Current State: _______c___
Attempts Left: 6
1/1 [==============================] - 0s 16ms/step
Guessing: c
Current State: _______c___
Attempts Left: 6
1/1 [==============================] - 0s 15ms/step
Guessing: c
Current State: _______c___
Attempts Left: 6
1/1 [==============================] - 0s 14ms/step
Guessing: c
Current State: _______c___
Attempts Left: 6
1/1 [==============================] - 0s 14ms/step
Guessing: c
Current State: _______c___
Attempts Left: 6
1/1 [==============================] - 0s 14ms/step
Guessing: c
Current State: _______c___
Attempts Left: 6
1/1 [==============================] - 0s 14ms/step
Guessing: c
Current State: _______c___
Attempts Left: 6
1/1 [======================

KeyboardInterrupt: 

In [15]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the trained model
model = load_model('hangman_dqn_model.h5')

# Dictionary for indexing characters for actions
chars = sorted(list(set('abcdefghijklmnopqrstuvwxyz')))
char_to_idx = {char: idx for idx, char in enumerate(chars)}
idx_to_char = {idx: char for char, idx in char_to_idx.items()}

# Hangman class
class Hangman:
    def __init__(self, word):
        self.word = word
        self.guesses = []
        self.max_attempts = 6
        self.attempts_left = self.max_attempts

    def guess(self, letter):
        self.guesses.append(letter)
        if letter not in self.word:
            self.attempts_left -= 1

    def is_word_guessed(self):
        return all(letter in self.guesses for letter in self.word)

    def get_current_state(self):
        return ''.join([letter if letter in self.guesses else '_' for letter in self.word])

    def get_remaining_attempts(self):
        return self.attempts_left

    def is_game_over(self):
        return self.attempts_left == 0 or self.is_word_guessed()

# Function to play Hangman using the trained DQN model
def play_hangman():
    word = random.choice(words_list)
    hangman = Hangman(word)
    state_size = len(chars)
    state = np.zeros(state_size)

    while not hangman.is_game_over():
        # Exclude previously guessed letters from the action space
        available_actions = [idx for idx in range(len(chars)) if idx_to_char[idx] not in hangman.guesses]

        # If all letters have been guessed, break the loop
        if not available_actions:
            break

        # Get the action (letter) from the DQN model
        action = model.predict(state.reshape(1, -1))
        action_idx = np.argmax(action[0][available_actions])
        letter = idx_to_char[available_actions[action_idx]]

        print(f"Guessing: {letter}")
        hangman.guess(letter)
        state = np.array([1 if idx_to_char[idx] in hangman.guesses else 0 for idx in range(state_size)])
        print(f"Current State: {hangman.get_current_state()}")
        print(f"Attempts Left: {hangman.get_remaining_attempts()}")

    if hangman.is_word_guessed():
        print(f"Congratulations! You guessed the word: {word}")
    else:
        print(f"Game Over! The word was: {word}")

if __name__ == "__main__":
    play_hangman()


1/1 [==============================] - 0s 36ms/step
Guessing: e
Current State: _e_____
Attempts Left: 6
1/1 [==============================] - 0s 37ms/step
Guessing: g
Current State: _e_____
Attempts Left: 5
1/1 [==============================] - 0s 15ms/step
Guessing: y
Current State: _e_____
Attempts Left: 4
1/1 [==============================] - 0s 14ms/step
Guessing: s
Current State: _e_____
Attempts Left: 3
1/1 [==============================] - 0s 15ms/step
Guessing: n
Current State: _e_n___
Attempts Left: 3
1/1 [==============================] - 0s 14ms/step
Guessing: m
Current State: _e_n___
Attempts Left: 2
1/1 [==============================] - 0s 15ms/step
Guessing: a
Current State: _e_n___
Attempts Left: 1
1/1 [==============================] - 0s 14ms/step
Guessing: i
Current State: _e_n_i_
Attempts Left: 1
1/1 [==============================] - 0s 14ms/step
Guessing: l
Current State: _e_n_i_
Attempts Left: 0
Game Over! The word was: deontic


In [1]:
with open('words_250000_train.txt', 'r') as f:
    words_list = f.read().splitlines()